In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',100)

In [2]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [3]:
from sklearn import set_config

set_config(display='diagram')

In [4]:
import warnings
warnings.filterwarnings('ignore')

## Read and understand the data

In [5]:
data = pd.read_csv('bank_marketing.csv')
data.head()

age        job marital_status    education  default housing loan  \
0   56  housemaid        married     basic.4y       no      no   no   
1   57   services        married  high.school  unknown      no   no   
2   37   services        married  high.school       no     yes   no   
3   40     admin.        married     basic.6y       no      no   no   
4   56   services        married  high.school       no      no  yes   

     contact month day_of_week  duration  campaign  pdays  previous  \
0  telephone   may         mon       261         1    999         0   
1  telephone   may         mon       149         1    999         0   
2  telephone   may         mon       226         1    999         0   
3  telephone   may         mon       151         1    999         0   
4  telephone   may         mon       307         1    999         0   

      poutcome  emp_var_rate  cons_price_idx  cons_conf_idx  euribor3m  \
0  nonexistent           1.1          93.994          -36.4      4.857   
1  nonexistent           1.1          93.994          -36.4      4.857   
2  nonexistent           1.1          93.994          -36.4      4.857   
3  nonexistent           1.1          93.994          -36.4      4.857   
4  nonexistent           1.1          93.994          -36.4      4.857   

   nr_employed response  
0       5191.0       no  
1       5191.0       no  
2       5191.0       no  
3       5191.0       no  
4       5191.0       no

In [6]:
X = data.drop('response', axis=1)
y = data['response']

In [7]:
from sklearn.compose import ColumnTransformer

num_attribs = X.select_dtypes('number').columns
cat_attribs = X.select_dtypes('O').columns

num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler()),
])

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))

preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])

In [8]:
preprocessing

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardize',
                                                  StandardScaler())]),
                                 Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed'],
      dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 Index(['job', 'marital_status', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'poutcome'],
      dtype='object'))])

In [9]:
X_transformed = preprocessing.fit_transform(X)

In [10]:
X_transformed.shape

(41188, 63)

In [11]:
X_transformed

array([[ 1.53303429,  0.01047142, -0.56592197, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.62899323, -0.42150051, -0.56592197, ...,  0.        ,
         1.        ,  0.        ],
       [-0.29018564, -0.12451981, -0.56592197, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 1.53303429, -0.26722482, -0.20490853, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.38152696,  0.70856893, -0.56592197, ...,  0.        ,
         1.        ,  0.        ],
       [ 3.26029527, -0.07438021,  0.15610492, ...,  1.        ,
         0.        ,  0.        ]])

In [12]:
from sklearn.ensemble import RandomForestClassifier

clf_pipe = make_pipeline(preprocessing, RandomForestClassifier())


In [13]:
clf_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['job', 'marital_status', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'poutcome'],
      dtype='object'))])),
                ('randomforestclassifier', RandomForestClassifier())])

In [14]:
clf_pipe.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['job', 'marital_status', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'poutcome'],
      dtype='object'))])),
                ('randomforestclassifier', RandomForestClassifier())])

In [15]:
clf_pipe.predict(X)

array(['no', 'no', 'no', ..., 'no', 'yes', 'no'], dtype=object)

In [16]:
import pickle

with open('ml_model.pkl', 'wb') as md:
    pickle.dump(clf_pipe, md)

In [17]:
with open('ml_model.pkl', 'rb') as fl:
    saved_model = pickle.load(fl)

In [18]:
saved_model.predict(X)

array(['no', 'no', 'no', ..., 'no', 'yes', 'no'], dtype=object)